In [9]:
# import libraries
import csv
import pandas as pd
import re
from pandas.core.frame import DataFrame
from pathlib import Path
import os
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
from statsmodels import robust
import json
import scipy
from scipy.stats import gmean, hmean
import seaborn as sns

### 3.1
Provide box plots including all successful latency measurements from
AS1.x data sets (one box per data set; ignore lost packets). Make sure
numerical values could be seen. What observations can be made, for
example, diﬀerences between sites? Were there diﬀerences in AS1.d_N_
and AS1.n*N ?

In [10]:
# exclude lost packets
def del_value_from_series(series,value):
    list = series.tolist()
    new_list = [x for x in list if x != value]
    new_series = pd.Series(new_list)
    return new_series

In [ ]:
plt.figure(figsize=(18,6))
for i in list(range(1,4)):
    df = pd.read_csv("./t3_processed/nameserver"+str(i)+"dig.csv")
    latencys = df.iloc[:,3]
    latencys_exclude_lost = del_value_from_series(latencys, float("+inf"))
    plt.subplot(1,3,i)
    plt.boxplot(latencys_exclude_lost);
    plt.title("boxplot of AS1.d"+str(i));
    plt.ylabel("latency[ms]")

plt.figure(figsize=(18,6))
for i in list(range(1,4)):
    df = pd.read_csv("./t3_processed/nameserver"+str(i)+"ping.csv")
    latencys = df.iloc[:,5]
    latencys_exclude_lost = del_value_from_series(latencys, float("+inf"))
    plt.subplot(1,3,i)
    plt.boxplot(latencys_exclude_lost);
    plt.title("boxplot of AS1.n"+str(i));
    plt.ylabel("latency[ms]")

plt.figure(figsize=(18,6))
for i in list(range(1,4)):
    df = pd.read_csv("./t3_processed/res"+str(i)+"ping.csv")
    latencys = df.iloc[:,5]
    latencys = del_value_from_series(latencys, "ms\n")
    latencys_exclude_lost = del_value_from_series(latencys, float("+inf"))
    latencys_exclude_lost = pd.to_numeric(latencys_exclude_lost)
    plt.subplot(1,3,i)
    plt.boxplot(latencys_exclude_lost);
    plt.title("boxplot of AS1.r"+str(i));
    plt.ylabel("latency[ms]")


plt.figure(figsize=(12,6))
for i in list(range(1,3)):
    df = pd.read_csv("./t3_processed/iperf"+str(i)+"ping.csv")
    latencys = df.iloc[:,5]
    latencys = del_value_from_series(latencys, "ms\n")
    # box plot would exclude inf automatically
    latencys_exclude_lost = del_value_from_series(latencys, float("+inf"))
    latencys_exclude_lost = pd.to_numeric(latencys_exclude_lost)
    plt.subplot(1,2,i)
    plt.boxplot(latencys_exclude_lost);
    plt.title("boxplot of AS1.i"+str(i));
    plt.ylabel("latency[ms]")

plt.figure(figsize=(12,6))
for i in list(range(1,3)):
    df = pd.read_csv("./t3_processed/iperf"+str(i)+"curl.csv")
    latencys = df.iloc[:,3]
    print(latencys.max())
    latencys = del_value_from_series(latencys, "ms\n")
    latencys_exclude_lost = del_value_from_series(latencys, float("+inf"))
    latencys_ms = [x*1000 for x in latencys_exclude_lost]
    plt.subplot(1,2,i)
    plt.boxplot(latencys_ms);
    plt.title("boxplot of AS1.icurl"+str(i));
    plt.ylabel("latency[ms]")



also consider the lost packets. One option
is to deﬁne all lost packets to have some maximum delay (like 2 seconds,
also any packet delayed more than 2 seconds would be shown as 2 s)

In [64]:
def lost_packets_convert(series, max_delay):
    list = series.tolist()
    new_list = []
    for x in list:
        if x == float("+inf"):
            new_list.append(max_delay)
        elif x >= max_delay:
            new_list.append(max_delay)
        else:
            new_list.append(x)
    new_series = pd.Series(new_list)

    return new_series

    


In [ ]:
# include lost packet
plt.figure(figsize=(18,6))
for i in list(range(1,4)):
    df = pd.read_csv("./t3_processed/nameserver"+str(i)+"dig.csv")
    latencys = df.iloc[:,3]
    latencys = del_value_from_series(latencys, "ms\n")
    latencys = pd.to_numeric(latencys)
    latencys = lost_packets_convert(latencys, 1000)  # lost packet to 2000ms
    plt.subplot(1,3,i)
    plt.boxplot(latencys);
    plt.title("boxplot of AS1.d"+str(i));
    plt.ylabel("latency[ms]")

plt.figure(figsize=(18,6))
for i in list(range(1,4)):
    df = pd.read_csv("./t3_processed/nameserver"+str(i)+"ping.csv")
    latencys = df.iloc[:,5]
    latencys = del_value_from_series(latencys, "ms\n")
    latencys = pd.to_numeric(latencys)
    latencys = lost_packets_convert(latencys, 1000)  # lost packet to 2000ms
    plt.subplot(1,3,i)
    plt.boxplot(latencys);
    plt.title("boxplot of AS1.n"+str(i));
    plt.ylabel("latency[ms]")

plt.figure(figsize=(18,6))
for i in list(range(1,4)):
    df = pd.read_csv("./t3_processed/res"+str(i)+"ping.csv")
    latencys = df.iloc[:,5]
    latencys = del_value_from_series(latencys, "ms\n")
    latencys = pd.to_numeric(latencys)
    latencys = lost_packets_convert(latencys, 1000)  # lost packet to 2000ms
    plt.subplot(1,3,i)
    plt.boxplot(latencys);
    plt.title("boxplot of AS1.r"+str(i));
    plt.ylabel("latency[ms]")

max_delays = [1000, 1000]
plt.figure(figsize=(12,6))
for i in list(range(1,3)):
    df = pd.read_csv("./t3_processed/iperf"+str(i)+"ping.csv")
    latencys = df.iloc[:,5]
    latencys = del_value_from_series(latencys, "ms\n")
    latencys = pd.to_numeric(latencys)
    latencys = lost_packets_convert(latencys, max_delays[i-1])  # lost packet to 2000ms
    plt.subplot(1,2,i)
    plt.boxplot(latencys);
    plt.title("boxplot of AS1.i"+str(i));
    plt.ylabel("latency[ms]")


plt.figure(figsize=(12,6))
for i in list(range(1,3)):
    df = pd.read_csv("./t3_processed/iperf"+str(i)+"curl.csv")
    latencys = df.iloc[:,3]
    latencys = del_value_from_series(latencys, "ms\n")
    latencys = pd.to_numeric(latencys)
    latencys_ms = pd.Series([x*1000 for x in latencys])
    latencys = lost_packets_convert(latencys_ms, max_delays[i-1])  # lost packet to 2000ms
    plt.subplot(1,2,i)
    plt.boxplot(latencys);
    plt.title("boxplot of AS1.icurl"+str(i));
    plt.ylabel("latency[ms]")


Characterise delay distributions according to ITU-T Y.1541 in a tabular
form for all AS1.x.

In [ ]:
table = [['server and command','delay characteristic']]
for filename in os.listdir("./t3_processed/"):
    file_path = os.path.join("./t3_processed/",filename)
    df = pd.read_csv(file_path)
    if 'ping' in filename:
        df = df[df['RTTs (ms)']!='ms\n']
        df = df[df['RTTs (ms)'] != 'inf']
        delay_spread = df['RTTs (ms)'].astype('float').quantile(0.75)-df['RTTs (ms)'].astype('float').quantile(0.25)
    elif 'dig' in filename:
        delay_spread = df['Query time (ms)'].astype('float').quantile(0.75)-df['Query time (ms)'].astype('float').quantile(0.25)
    elif 'curl' in filename and 'th' not in filename:
        delay_spread = df['TCP latency (s)'].astype('float').quantile(0.75)-df['TCP latency (s)'].astype('float').quantile(0.25)
    else:
        continue
    table.append([filename.split('.csv')[0], delay_spread])
print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

### 3.2
Plot time series of each data set AS1.x. Consider appropriate scaling for
comparison. Any observations for e.g. diurnal patterns?

In [ ]:
# research server
plt.figure(figsize=(16,9))
df = pd.read_csv("./t3_processed/res1ping.csv")
timestamps = df.loc[:,'Timestamp']
delays = df.loc[:,'RTTs (ms)']
plt.plot(timestamps, delays)
df = pd.read_csv("./t3_processed/res2ping.csv")
df = df[df['RTTs (ms)']!='ms\n']
timestamps = pd.to_numeric(df.loc[:,'Timestamp'])
delays = pd.to_numeric(df.loc[:,'RTTs (ms)'])
plt.plot(timestamps, delays)
df = pd.read_csv("./t3_processed/res3ping.csv")
df = df[df['RTTs (ms)']!='ms\n']
timestamps = pd.to_numeric(df.loc[:,'Timestamp'])
delays = pd.to_numeric(df.loc[:,'RTTs (ms)'])
plt.plot(timestamps, delays)
plt.legend(["research server1","research server2","research server3"])
plt.xlabel("measurement time[s]")
plt.ylabel("delay[ms]")
plt.title('time series')

# # nameserver
plt.figure(figsize=(16,9))
df = pd.read_csv("./t3_processed/nameserver1ping.csv")
timestamps = df.loc[:,'Timestamp']
delays = df.loc[:,'RTTs (ms)']
plt.plot(timestamps, delays)
df = pd.read_csv("./t3_processed/nameserver2ping.csv")
timestamps = df.loc[:,'Timestamp']
delays = df.loc[:,'RTTs (ms)']
plt.plot(timestamps, delays)
df = pd.read_csv("./t3_processed/nameserver3ping.csv")
timestamps = df.loc[:,'Timestamp']
delays = df.loc[:,'RTTs (ms)']
plt.plot(timestamps, delays)
plt.legend(["name server1","name server2","name server3"])
plt.xlabel("measurement time[s]")
plt.ylabel("delay[ms]")
plt.title('time series')

# # iperf server
plt.figure(figsize=(16,9))
df = pd.read_csv("./t3_processed/iperf1ping.csv")
df = df[df['RTTs (ms)']!='ms\n']
timestamps = pd.to_numeric(df.loc[:,'Timestamp'])
delays = pd.to_numeric(df.loc[:,'RTTs (ms)'])
plt.plot(timestamps, delays)
df = pd.read_csv("./t3_processed/iperf2ping.csv")
df = df[df['RTTs (ms)']!='ms\n']
timestamps = pd.to_numeric(df.loc[:,'Timestamp'])
delays = pd.to_numeric(df.loc[:,'RTTs (ms)'])
plt.plot(timestamps, delays)
plt.legend(["iperf server1","iperf server2"])
plt.xlabel("measurement time[s]")
plt.ylabel("delay[ms]")
plt.title('time series')


# comparison


Select AS1.i2 and minimum two other most interesting data sets from
AS1.x. Make an autocorrelation plot. Any observations?

In [ ]:
from pandas.plotting import autocorrelation_plot

# i2
df = pd.read_csv("./t3_processed/iperf2ping.csv")
df = df[df['RTTs (ms)']!='ms\n']
delays = pd.to_numeric(df.loc[:,'RTTs (ms)'])
delays = del_value_from_series(delays, float("+inf"))  # ignore inf value
plt.figure(figsize=(16,9))
autocorrelation_plot(delays)
# plt.ylim([-0.25,0.25])
plt.title("autocorrelation plot of iperf server 2")



df = pd.read_csv("./t3_processed/res3ping.csv")
df = df[df['RTTs (ms)']!='ms\n']
delays = pd.to_numeric(df.loc[:,'RTTs (ms)'])
delays = del_value_from_series(delays, float("+inf"))  # ignore inf value
plt.figure(figsize=(16,9))
autocorrelation_plot(delays)
plt.title("autocorrelation plot of research server 3")



df = pd.read_csv("./t3_processed/nameserver2ping.csv")
df = df[df['RTTs (ms)']!='ms\n']
delays = pd.to_numeric(df.loc[:,'RTTs (ms)'])
delays = del_value_from_series(delays, float("+inf"))  # ignore inf value
plt.figure(figsize=(16,9))
autocorrelation_plot(delays)
plt.title("autocorrelation plot of name server 2")






### 3.3

In [87]:
df_one = pd.read_csv("./t3_processed/th_one.csv")
df_one_r = pd.read_csv("./t3_processed/th_one_r.csv")
df_two = pd.read_csv("./t3_processed/th_two.csv")
df_two_r = pd.read_csv("./t3_processed/th_two_r.csv")

In [ ]:
# remove fail measurements
# one
receive_br = df_one.loc[:,'Receive bitrate (bps)']
sent_br = df_one.loc[:,'Sent bitrate (bps)']
receive_br_filtered = receive_br[receive_br != -1]
sent_br_filtered = sent_br[sent_br != -1]
plt.figure(figsize=(16,9))
plt.subplot(1,2,1)
plt.boxplot(receive_br_filtered)
plt.ylabel("bitrate[bps]")
plt.title("iperf server 1 regular mode receive bitrate")
plt.subplot(1,2,2)
plt.boxplot(sent_br_filtered)
plt.ylabel("bitrate[bps]")
plt.title("iperf server 1 regular mode sent bitrate")

# one r
receive_br = df_one_r.loc[:,'Receive bitrate (bps)']
sent_br = df_one_r.loc[:,'Sent bitrate (bps)']
receive_br_filtered = receive_br[receive_br != -1]
sent_br_filtered = sent_br[sent_br != -1]
plt.figure(figsize=(16,9))
plt.subplot(1,2,1)
plt.boxplot(receive_br_filtered)
plt.ylabel("bitrate[bps]")
plt.title("iperf server 1 reverse mode receive bitrate")
plt.subplot(1,2,2)
plt.boxplot(sent_br_filtered)
plt.ylabel("bitrate[bps]")
plt.title("iperf server 1 reverse mode sent bitrate")

# two
receive_br = df_two.loc[:,'Receive bitrate (bps)']
sent_br = df_two.loc[:,'Sent bitrate (bps)']
receive_br_filtered = receive_br[receive_br != -1]
sent_br_filtered = sent_br[sent_br != -1]
plt.figure(figsize=(16,9))
plt.subplot(1,2,1)
plt.boxplot(receive_br_filtered)
plt.ylabel("bitrate[bps]")
plt.title("iperf server 2 regular mode receive bitrate")
plt.subplot(1,2,2)
plt.boxplot(sent_br_filtered)
plt.ylabel("bitrate[bps]")
plt.title("iperf server 2 regular mode sent bitrate")

# two r
receive_br = df_two_r.loc[:,'Receive bitrate (bps)']
sent_br = df_two_r.loc[:,'Sent bitrate (bps)']
receive_br_filtered = receive_br[receive_br != -1]
sent_br_filtered = sent_br[sent_br != -1]
plt.figure(figsize=(16,9))
plt.subplot(1,2,1)
plt.boxplot(receive_br_filtered)
plt.ylabel("bitrate[bps]")
plt.title("iperf server 2 reverse mode receive bitrate")
plt.subplot(1,2,2)
plt.boxplot(sent_br_filtered)
plt.ylabel("bitrate[bps]")
plt.title("iperf server 2 reverse mode sent bitrate")

tabulate

In [89]:
from tabulate import tabulate

In [90]:
def calculate_tabulate(name,series):
    series = pd.Series(series)
    mean = series.mean()
    geo_mean = gmean(series)
    har_mean = hmean(series)
    med = series.median()
    return [name, round(mean), round(geo_mean), round(har_mean), round(med)]

In [91]:
table = [[' ', 'Mean[bps]', 'Geometric mean[bps]', 'Harmonic mean[bps]', 'Median[bps]']]
# one
receive_br = df_one.loc[:,'Receive bitrate (bps)']
sent_br = df_one.loc[:,'Sent bitrate (bps)']
receive_br_filtered = receive_br[receive_br != -1]
sent_br_filtered = sent_br[sent_br != -1]
table.append(calculate_tabulate('iperf 1 regular receive',receive_br_filtered))
table.append(calculate_tabulate('iperf 1 regular sent',sent_br_filtered))

receive_br = df_one_r.loc[:,'Receive bitrate (bps)']
sent_br = df_one_r.loc[:,'Sent bitrate (bps)']
receive_br_filtered = receive_br[receive_br != -1]
sent_br_filtered = sent_br[sent_br != -1]
table.append(calculate_tabulate('iperf 1 reverse receive',receive_br_filtered))
table.append(calculate_tabulate('iperf 1 reverse sent',sent_br_filtered))

receive_br = df_two.loc[:,'Receive bitrate (bps)']
sent_br = df_two.loc[:,'Sent bitrate (bps)']
receive_br_filtered = receive_br[receive_br != -1]
sent_br_filtered = sent_br[sent_br != -1]
table.append(calculate_tabulate('iperf 2 regular receive',receive_br_filtered))
table.append(calculate_tabulate('iperf 2 regular sent',sent_br_filtered))

receive_br = df_two_r.loc[:,'Receive bitrate (bps)']
sent_br = df_two_r.loc[:,'Sent bitrate (bps)']
receive_br_filtered = receive_br[receive_br != -1]
sent_br_filtered = sent_br[sent_br != -1]
table.append(calculate_tabulate('iperf 2 reverse receive',receive_br_filtered))
table.append(calculate_tabulate('iperf 2 reverse sent',sent_br_filtered))

print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))


╒═════════════════════════╤═════════════╤═══════════════════════╤══════════════════════╤═══════════════╕
│                         │   Mean[bps] │   Geometric mean[bps] │   Harmonic mean[bps] │   Median[bps] │
╞═════════════════════════╪═════════════╪═══════════════════════╪══════════════════════╪═══════════════╡
│ iperf 1 regular receive │   363518616 │             355641867 │            347710530 │     355832598 │
├─────────────────────────┼─────────────┼───────────────────────┼──────────────────────┼───────────────┤
│ iperf 1 regular sent    │   366145501 │             358310375 │            350420723 │     358568519 │
├─────────────────────────┼─────────────┼───────────────────────┼──────────────────────┼───────────────┤
│ iperf 1 reverse receive │   580170798 │             577708951 │            573382668 │     592577623 │
├─────────────────────────┼─────────────┼───────────────────────┼──────────────────────┼───────────────┤
│ iperf 1 reverse sent    │   582232356 │             5

### 3.4

time series

In [ ]:
df_one = pd.read_csv("./t3_processed/th_one.csv")
timestamps = df_one.loc[:,'Timestamp']
timestamps_filtered = timestamps[ timestamps != -1]
receive_br = df_one.loc[:,'Receive bitrate (bps)']
sent_br = df_one.loc[:,'Sent bitrate (bps)']
receive_br_filtered = receive_br[receive_br != -1]
sent_br_filtered = sent_br[sent_br != -1]
plt.figure(figsize=(16,9))
plt.plot(timestamps_filtered, receive_br_filtered)
plt.plot(timestamps_filtered, sent_br_filtered)
plt.xlabel("time")
plt.ylabel("bitrate[bps]")
plt.legend(["receive bitrate","sent bitrate"])
plt.title("time series is iperf 1 regular mode")


df_one_r = pd.read_csv("./t3_processed/th_one_r.csv")
timestamps = df_one_r.loc[:,'Timestamp']
timestamps_filtered = timestamps[ timestamps != -1]
receive_br = df_one_r.loc[:,'Receive bitrate (bps)']
sent_br = df_one_r.loc[:,'Sent bitrate (bps)']
receive_br_filtered = receive_br[receive_br != -1]
sent_br_filtered = sent_br[sent_br != -1]
plt.figure(figsize=(16,9))
plt.plot(timestamps_filtered, receive_br_filtered)
plt.plot(timestamps_filtered, sent_br_filtered)
plt.xlabel("time")
plt.ylabel("bitrate[bps]")
plt.legend(["receive bitrate","sent bitrate"])
plt.title("time series is iperf 1 reverse mode")


df_two = pd.read_csv("./t3_processed/th_two.csv")
timestamps = df_two.loc[:,'Timestamp']
timestamps_filtered = timestamps[ timestamps != -1]
receive_br = df_two.loc[:,'Receive bitrate (bps)']
sent_br = df_two.loc[:,'Sent bitrate (bps)']
receive_br_filtered = receive_br[receive_br != -1]
sent_br_filtered = sent_br[sent_br != -1]
plt.figure(figsize=(16,9))
plt.plot(timestamps_filtered, receive_br_filtered)
plt.plot(timestamps_filtered, sent_br_filtered)
plt.xlabel("time")
plt.ylabel("bitrate[bps]")
plt.legend(["receive bitrate","sent bitrate"])
plt.title("time series is iperf 2 regular mode")


df_two_r = pd.read_csv("./t3_processed/th_two_r.csv")
timestamps = df_two_r.loc[:,'Timestamp']
timestamps_filtered = timestamps[ timestamps != -1]
receive_br = df_two_r.loc[:,'Receive bitrate (bps)']
sent_br = df_two_r.loc[:,'Sent bitrate (bps)']
receive_br_filtered = receive_br[receive_br != -1]
sent_br_filtered = sent_br[sent_br != -1]
plt.figure(figsize=(16,9))
plt.plot(timestamps_filtered, receive_br_filtered)
plt.plot(timestamps_filtered, sent_br_filtered)
plt.xlabel("time")
plt.ylabel("bitrate[bps]")
plt.legend(["receive bitrate","sent bitrate"])
plt.title("time series is iperf 2 reverse mode")




autocorrelation plot

In [ ]:
df_one = pd.read_csv("./t3_processed/th_one.csv")
receive_br = df_one.loc[:,'Receive bitrate (bps)']
sent_br = df_one.loc[:,'Sent bitrate (bps)']
receive_br_filtered = receive_br[receive_br != -1]
sent_br_filtered = sent_br[sent_br != -1]

plt.figure(figsize=(10,8))
plt.title("autocorrelation plot of iperf 1 regular mode")
autocorrelation_plot(receive_br_filtered)
# plt.figure(figsize=(10,8))
autocorrelation_plot(sent_br_filtered)
plt.legend(['receive_br', 'sent_br'])


df_one_r = pd.read_csv("./t3_processed/th_one_r.csv")
receive_br = df_one_r.loc[:,'Receive bitrate (bps)']
sent_br = df_one_r.loc[:,'Sent bitrate (bps)']
receive_br_filtered = receive_br[receive_br != -1]
sent_br_filtered = sent_br[sent_br != -1]

plt.figure(figsize=(10,8))
plt.title("autocorrelation plot of iperf 1 reverse mode")
autocorrelation_plot(receive_br_filtered)
# plt.figure(figsize=(10,8))
autocorrelation_plot(sent_br_filtered)
plt.legend(['receive_br', 'sent_br'])



df_two = pd.read_csv("./t3_processed/th_two.csv")
receive_br = df_two.loc[:,'Receive bitrate (bps)']
sent_br = df_two.loc[:,'Sent bitrate (bps)']
receive_br_filtered = receive_br[receive_br != -1]
sent_br_filtered = sent_br[sent_br != -1]

plt.figure(figsize=(10,8))
plt.title("autocorrelation plot of iperf 2 regular mode")
autocorrelation_plot(receive_br_filtered)
# plt.figure(figsize=(10,8))
autocorrelation_plot(sent_br_filtered)
plt.legend(['receive_br', 'sent_br'])



df_two_r = pd.read_csv("./t3_processed/th_two_r.csv")
receive_br = df_two_r.loc[:,'Receive bitrate (bps)']
sent_br = df_two_r.loc[:,'Sent bitrate (bps)']
receive_br_filtered = receive_br[receive_br != -1]
sent_br_filtered = sent_br[sent_br != -1]

plt.figure(figsize=(10,8))
plt.title("autocorrelation plot of iperf 2 reverse mode")
autocorrelation_plot(receive_br_filtered)
# plt.figure(figsize=(10,8))
autocorrelation_plot(sent_br_filtered)
plt.legend(['receive_br', 'sent_br'])


